In [ ]:
# ## 1 Užduotis

# Panaudokite Grid Search, Random Search ir Bayes optimizavimą vienam iš savo anksčiau naudotų datasetų, 
# jog surastumėte didžiausią tikslumą ir geriausius hiperparametrus

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd
from sklearn.preprocessing import StandardScaler
from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll.base import scope
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [3]:
data = pd.read_csv('data.csv', sep=',')

data = data.drop(['Unnamed: 32', 'id'], axis=1)


data['diagnosis'] = data['diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

X = data.drop('diagnosis', axis=1)
y = data['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 50, 100, 200], # medziu kiekis
    'max_depth': [None, 10, 20], # sprendimu kiekis medyje
    'min_samples_split': [2, 5, 10] # minimalus pavyzdžių kiekis
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X,y)


print(f'Geriausi parametrai: {grid_search.best_params_}')
print(f'Geriausias tikslumas: {grid_search.best_score_}')




Geriausi parametrai: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 10}
Geriausias tikslumas: 0.9666666666666666


In [18]:
skaler = StandardScaler()
X_train = skaler.fit_transform(X_train)
X_test = skaler.transform(X_test)

def objective(params):
    model = RandomForestClassifier(**params)
    return -cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

models_params = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 10, 200, 1)), # medziu kiekis
    'max_depth': hp.choice('max_depth', [None, 10, 20]), # sprendimu kiekis medyje
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 20, 1)) # minimalus pavyzdžių kiekis
}

trials = Trials()
best = fmin(fn=objective, space=models_params, algo=tpe.suggest, max_evals=100, trials=trials )
print(f"Geriausi parametrai: {best}")

100%|██████████| 100/100 [01:11<00:00,  1.39trial/s, best loss: -0.9666356155876417]
Geriausi parametrai: {'max_depth': 1, 'min_samples_split': 3.0, 'n_estimators': 110.0}


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = RandomForestClassifier(max_depth=1, min_samples_split=3, n_estimators=110)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.958041958041958
